# TASK length and TASK ZERO all test

# Test Description
This script will test the functionality of number of Tasks creation and Task zero command.
As per datasheet maximum 64K tasks can be created. you can enter number of task creation in the variable name 'tasklen'. 
After tak creation and download, all tasks can be deleted using :TASK:ZERO:ALL command.
So if test is pass if task creation and deletion is perfect.
This test run for channel1 only with sampling rate 1GHz.


# Test Equipment
1. Tabor Proteus 9484M
2. Keysight Oscilloscope MSO9254A
3. 4 SMA to BNC Cable

# Hardware Connection
    Proteus Module    --->     Oscilloscope
    Channel 1         --->      Channel 1
    Channel 2         --->      Channel 2
    Channel 3         --->      Channel 3
    Channel 4         --->      Channel 4


# Test Procedure
1. Connect all the channels of Proteus module with the concerned channel of oscilloscope.
2. Run the script and observe output on scope.

Proteus release - Aurora



In [1]:
# Import required Libraries
import os
import sys
import tempfile
import webbrowser
srcpath = os.path.realpath('SourceFiles')
sys.path.append(srcpath)
from teproteus import TEProteusAdmin as TepAdmin
from teproteus import TEProteusInst as TepInst
from teproteus_functions_v3 import connect
from teproteus_functions_v3 import disconnect
from teproteus_functions_v3 import set_lib_dir_path
from teproteus_functions_v3 import get_cpatured_header
from teproteus_functions_v3 import gauss_env
from teproteus_functions_v3 import iq_kernel
from teproteus_functions_v3 import pack_kernel_data
from teproteus import TEProteusAdmin, TEProteusInst
from tevisainst import TEVisaInst
#matplotlib notebook
import pyvisa as visa
import numpy as np
import time
import ipywidgets as widgets
from IPython.core.debugger import set_trace
from scipy.signal import chirp, sweep_poly
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from scipy import signal
import math
import pdb

In [2]:
# Connect to instrument
connection = 'LAN' #input ("Do you wish to connect via LAN/PXI/LOCAL/USB ")
if connection == 'PXI':   
    # not setting this command means DLL are taken from SYS32
    #set_lib_dir_path(r'D:\Projects\ProteusAwg_Anabelle\x64\Debug') 
    # for service connection enter instrument IP
    # for PXI DLL connection enter the module slot number with Auto=False
    # when Auto=True the lowest module will be connected
    inst=connect("2",Auto=False)
    
elif connection == 'LAN':
    proteus_addr = 'TCPIP::192.90.70.22::5025::SOCKET'
    try:
        inst = TEVisaInst(proteus_addr)
    except TEVisaInst.Error as  ex1:
        print('Couldn\'t connect to \'%s\', exiting now...' % proteus_addr)
        sys.exit()
        
elif connection == 'USB':
    proteus_addr = 'USB0::0x2A8D::0x900E::MY55490134::INSTR'
    try:
        inst = TEVisaInst(proteus_addr)
    except TEVisaInst.Error as  ex1:
        print('Couldn\'t connect to \'%s\', exiting now...' % proteus_addr)
        sys.exit()
        
elif connection == 'LOCAL':
    proteus_addr = 'TCPIP::127.0.0.1::5025::SOCKET'
    try:
        inst = TEVisaInst(proteus_addr)
    except TEVisaInst.Error as  ex1:
        print('Couldn\'t connect to \'%s\', exiting now...' % proteus_addr)
        sys.exit()
        
else:
    print("Please select the conection method first")
# Get the instrument's *IDN
resp = inst.send_scpi_query('*IDN?')
print('Connected to: ' + resp)
inst.default_paranoia_level = 2


Connected to: Tabor Electronics,P9484M,000002232771,1.238.6 --Tabor Electronics,P9484M,000000000000,1.238.6 --slot#: 2, slot#: 4, 


In [3]:
# Get the model:
model_name = inst.send_scpi_query('SYST:INF:MODel?')
print('Model: {0} '.format(model_name))

# Get model dependant parameters:
if model_name.startswith('P948'):
    bpp = 2
    max_dac = 65535
    wpt_type = np.uint16
    offset_factor = 1
elif model_name.startswith('P908'):
    bpp = 1
    max_dac = 255
    wpt_type = np.uint8
    offset_factor = 1
else:
    bpp = 2
    max_dac = 65535
    wpt_type = np.uint16
    offset_factor = 2
    
half_dac = max_dac / 2.0    
# Get the DAC mode (8 bits or 16 bits)

resp = inst.send_scpi_query(':SYST:INF:DAC?')
if resp == 'M0': 
    dac_mode=16 
else: dac_mode=8
   
print('DAC {0} bits'.format(dac_mode))

Model: P9484M 
DAC 16 bits


In [4]:
# Several initializations ..
inst.send_scpi_cmd('*CLS; *RST')
#proteus.send_scpi_cmd(':INST:CHAN 2')
inst.send_scpi_cmd(':TRAC:DEL:ALL') # Delete all segments of the programmable channel's DDR.
resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)
# Get number of channels
num_channels = inst.send_scpi_query(":INST:CHAN? MAX")
print("Number of channels: " + num_channels)
#num_channels = int(resp)

0, no error
Number of channels: 4


In [5]:
FREQ = 1000
inst.send_scpi_cmd(":FREQ:RAST {0}MHz".format(FREQ))
resp = inst.send_scpi_query(":FREQ:RAST?")
freq = float(resp)
print ("DAC Generate Freq:{0}".format(freq))

DAC Generate Freq:1000000000.0


In [6]:
scope_addr2=  'USB0::0x2A8D::0x900E::MY55490134::INSTR'

# connect to scope via USB
try:
    resourceManager = visa.ResourceManager()   # Create a connection (session) to the instrument
    scope = resourceManager.open_resource(scope_addr2)
except visa.Error as ex2:
        print('Couldn\'t connect to \'%s\', exiting now...' % scope_addr2)
        sys.exit()
    
    
    
## scope acquisition 
# Send *IDN? and read the response
scope.write('*RST?')
scope.write('*IDN?')
idn = scope.read()
print('*IDN? returned: %s' % idn.rstrip('\n'))

for i in range(4):
    scope.write(':CHANnel{}:DISPlay OFF'.format(i+1)) 
    scope.write(':MEASure:CLEar')


*IDN? returned: KEYSIGHT TECHNOLOGIES,MSO9254A,MY55490134,06.40.01101


In [7]:
tasklen = 100 

# Build sine wave on channel 1
segnum = 1
seglen = 1024
ncycles1 = 1
# cyclelen = seglen
cyclelen = (seglen/ncycles1)
waves = [ None for _ in range(3)]
# sin wave:
x = np.linspace(start=0, stop=2 * np.pi * ncycles1, num=seglen, endpoint=False)
y = (np.sin(x) + 1.0) * half_dac
y = np.round(y)
y = np.clip(y, 0, max_dac)
waves[0] = y.astype(wpt_type)
#         plt.plot(x,y)
#         print("************* WAVEFORM*****************")
# Download sine wave to channel 1
for ii in range(1):
    ichan = ii
    channb = ichan + 1
    segnum = ii +1
wav = waves[0]
#         print('Download wave to segment {} of channel {}'.format(segnum,channel_select[in_channelselect]))    
# Select channel
inst.send_scpi_cmd(':INST:CHAN {}'.format(1))    
# Define segment
inst.send_scpi_cmd(':TRAC:DEF {},{}'.format(segnum, seglen))
# Select the segment
inst.send_scpi_cmd(':TRAC:SEL {}'.format(segnum))
#     proteus.send_scpi_cmd(':VOLT:OFFS 0.2')
inst.send_scpi_cmd(':VOLT 0.5')
# Increase the timeout before writing binary-data:
inst.timeout = 30000    
# Select the segment
cmd = ':TRAC:SEL {0}'.format(segnum)
inst.send_scpi_cmd(cmd)        
# Send the binary-data:
inst.write_binary_data(':TRAC:DATA', wav)
resp = inst.send_scpi_query(':SYST:ERR?')
resp = resp.rstrip()
if not resp.startswith('0'):
    print('ERROR: "{}" after writing binary values'.format(resp))
# Play the specified segment at the selected channel:
cmd = ':SOUR:FUNC:MODE:SEGM {}'.format(segnum)
inst.send_scpi_cmd(cmd)
# Turn on the output of the selected channel:
inst.send_scpi_cmd(':OUTP ON')    
resp = inst.send_scpi_query(':SYST:ERR?')
#         print(resp)
#         print()
        
scope.write('*CLS;:DISPlay:CGRade:LEVels ')
scope.write('AUTOscale')
time.sleep(3)
scope.write(':MEASure:VPP CHANnel{}'.format(1))
time.sleep(2)
scope.write(':MEASure:RESults?')
result = scope.read()
peak_peak_volt1= float(result.split(',')[1])
# print(peak_peak_volt1)
scope.write(':MEASure:FREQuency CHANnel{}'.format(1))
scope.write(':MEASure:RESults?')
time.sleep(1)
result = scope.read()
meas_freq1 = float(result.split(',')[2])/1e6
# print(meas_freq1)  
        
        
time.sleep(3)


      
scope.write(':CHANnel{}:DISPlay ON'.format(1))
inst.send_scpi_cmd(':TASK:COMP:LENG {0}'.format(tasklen))
for j in range(tasklen):
    curr_task = j+1
    inst.send_scpi_cmd(':TASK:COMP:SEL {}'.format(curr_task))
    inst.send_scpi_cmd(':TASK:COMP:TYPE SING')
    inst.send_scpi_cmd(':TASK:COMP:ENAB NONE')
    inst.send_scpi_cmd(':TASK:COMP:DEL 1000') 
    inst.send_scpi_cmd(':TASK:COMP:SEGM {0}'.format(1))                  # play segment 2 SQP
    if curr_task==tasklen:
        inst.send_scpi_cmd(':TASK:COMP:NEXT1 {0}'.format(1))
    else:
        inst.send_scpi_cmd(':TASK:COMP:NEXT1 {0}'.format(curr_task+1))

    print('Task no {} is created successfully'.format(curr_task))

             
inst.send_scpi_cmd(':TASK:COMP:WRIT')
inst.send_scpi_cmd(':INST:CHAN {}'.format(1))
inst.send_scpi_cmd(':FUNC:MODE TASK')
inst.send_scpi_cmd(':OUTP ON')

        
        
scope.write('*CLS;:DISPlay:CGRade:LEVels ')
scope.write('AUTOscale')
time.sleep(3)
scope.write(':MEASure:VPP CHANnel{}'.format(1))
time.sleep(2)
scope.write(':MEASure:RESults?')
result = scope.read()
peak_peak_volt2= float(result.split(',')[1])
# print(peak_peak_volt2)
scope.write(':MEASure:FREQuency CHANnel{}'.format(1))
scope.write(':MEASure:RESults?')
time.sleep(1)
result = scope.read()
meas_freq2 = float(result.split(',')[2])/1e6
# print(meas_freq2)
        
        
            
inst.send_scpi_cmd(':INST:CHAN {}'.format(1))     
inst.send_scpi_cmd(':TASK:ZERO:ALL')
time.sleep(3)
#         scope.write('*CLS;:DISPlay:CGRade:LEVels ')
#         scope.write('AUTOscale')
time.sleep(3)
scope.write(':MEASure:VPP CHANnel{}'.format(1))
time.sleep(2)
scope.write(':MEASure:RESults?')
result = scope.read()
peak_peak_volt3= float(result.split(',')[1])
# print(peak_peak_volt3)
scope.write(':MEASure:FREQuency CHANnel{}'.format(1))
scope.write(':MEASure:RESults?')
time.sleep(1)
result = scope.read()
meas_freq3 = float(result.split(',')[2])/1e6
# print(meas_freq3)

if 0.95<meas_freq1<0.99 and 0.49<meas_freq2<0.55 and 0.95<meas_freq1<0.99:
    if 0.49<peak_peak_volt1<0.54 and 0.49<peak_peak_volt2<0.54 and 0.49<peak_peak_volt3<0.54:
        print('\t***Test Pass for Task table zero command***'.format(1))
    else:
        print('\t***Test Fail for Channel {} due to peak value***'.format(1))
else:
    print('\t***Test Fail for Channel {} due to frequency***'.format(1))

inst.send_scpi_cmd(':OUTP OFF')
scope.write(':CHANnel{}:DISPlay OFF'.format(1))
        
# close inst connection
inst.close_instrument()
disconnect()

# Close the connection to the instrument
scope.close()
resourceManager.close()        

Task no 1 is created successfully
Task no 2 is created successfully
Task no 3 is created successfully
Task no 4 is created successfully
Task no 5 is created successfully
Task no 6 is created successfully
Task no 7 is created successfully
Task no 8 is created successfully
Task no 9 is created successfully
Task no 10 is created successfully
Task no 11 is created successfully
Task no 12 is created successfully
Task no 13 is created successfully
Task no 14 is created successfully
Task no 15 is created successfully
Task no 16 is created successfully
Task no 17 is created successfully
Task no 18 is created successfully
Task no 19 is created successfully
Task no 20 is created successfully
Task no 21 is created successfully
Task no 22 is created successfully
Task no 23 is created successfully
Task no 24 is created successfully
Task no 25 is created successfully
Task no 26 is created successfully
Task no 27 is created successfully
Task no 28 is created successfully
Task no 29 is created success